In [1]:
import gym
import gym_Aircraft
from gym_Aircraft.envs.Aircraft_env import (
    NOT_DONE,            # 0
    CRASHED,             # 1
    AVOIDED_TIMEOUT,     # 2
    AVOIDED_IN_ADVANCE,  # 3
)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils

import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
import random

In [2]:
env = gym.make("acav-v0")
# env = gym.make("CartPole-v1")
# env.action_space.seed(960501) 

/home/hyogun/anaconda3/envs/collision_avoidance/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
dim_obs = env.observation_space.shape[0]
dim_act = env.action_space.n
max_episode = 5000
max_replay = 50000
batch_size = 256 
gamma = 1.  # ㅋㅋㅋ .9에서 1로 바꾸니까 갑자기 잘됨
eps = 1.  # 얘도 1로 시작해야 되네
eps_decay = .99
eps_decay_step = 10
learning_rate = 1e-7  # 학습하다가 발산하면 얘부터 만져봐야 됨
max_grad_norm = 10

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(dim_obs, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, dim_act),
            nn.LeakyReLU(.5),  # 와 이거 혁신이네
        )
        
    def forward(self, *input):
        return self.model(*input).squeeze()

In [5]:
net = DQN().to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

memory = []
record = {'score': [], 'average loss': [], 'actions': []}

global_step = 0
train_step = 0
for episode in range(max_episode):
    s0 = env.reset()
    done = False
    net_reward = 0
    episode_loss = 0
    episode_norm = 0
    episode_step = 0
    episode_actions = []
    
    while not done:
        if train_step % eps_decay_step == eps_decay_step - 1:
            eps *= eps_decay
            eps = max(.01, eps)
            
        if random.random() > eps:
            a = np.argmax(net(torch.from_numpy(s0).float().to(device)).detach().cpu())
            a = int(a) 
        else:
            a = env.action_space.sample()
        
        # 의도적인 탐험
#         if episode < 50: a = 1
#         elif episode < 100: a = 2
#         elif episode < 150:
#             a = 1 if episode_step < 50 else 0
#         elif episode < 200:
#             a = 2 if episode_step < 70 else 0
            
        s1, reward, done, _ = env.step(a)
        s0 = s1
        net_reward += reward
        if not done: 
            episode_actions.append(a)
        
        memory.append([s0, a, reward, s1, done])
        if len(memory) > max_replay:
            del memory[0]
        
        if global_step == max_replay - 1:
            print('Training now begins')
        
# https://stackoverflow.com/questions/52770780/why-is-my-deep-q-net-and-double-deep-q-net-unstable
        if len(memory) == max_replay and global_step % 2 == 0:  # soft update
            y_batch = np.zeros(batch_size)
            s0_batch = np.zeros((batch_size, dim_obs))
            for j, sample in enumerate(random.sample(memory, batch_size)):
                mem_s0, mem_a, mem_reward, mem_s1, is_terminal = sample
                s0_batch[j] = mem_s0
                if is_terminal is NOT_DONE:
                    # y_j = r_j + gamma * max_a' Q(s_j+1, a')
                    y_batch[j] = mem_reward + gamma * torch.max(net(torch.from_numpy(mem_s1).float().to(device)).detach()).cpu().numpy()
                else:
                    y_batch[j] = mem_reward
            # optimize Q w.r.t ||max_a Q(s, a) - y||
            loss = criterion(
                torch.max(net(torch.from_numpy(s0_batch).float().to(device)), 1)[0],
                torch.from_numpy(y_batch).float().to(device)
            )
            loss.backward()
            episode_norm += torch_utils.clip_grad_norm_(net.parameters(), max_grad_norm)
            optimizer.step()
            train_step += 1
            episode_loss += loss
        episode_step += 1
        global_step += 1
    
    # episode log
    # score가 아니라 얼마나 잘 깨는지 봐야 됨
    # score는 단순히 사람 편하자고 계산하는 수임
    print(f'Epi. {episode+1:4d} score: {net_reward:4.0f}', end=' ')
    if len(memory) == max_replay:
        average_episode_loss = episode_loss/episode_step
        record['score'].append(net_reward)
        record['average loss'].append(average_episode_loss)
        record['actions'].append(episode_actions)
        c = Counter(episode_actions)
        print(
            f'E[L]: {average_episode_loss:6.1f} '
            f'e: {eps:.4f} norm: {episode_norm/episode_step:.0f} '
            f'actions: {c[0]:3d} {c[1]:3d} {c[2]:3d} {done}'
        )
    else:
        print()

Epi.    1 score:  -13 
Epi.    2 score:   -7 
Epi.    3 score:   -9 
Epi.    4 score:  -13 
Epi.    5 score:   -8 
Epi.    6 score:  -23 
Epi.    7 score:   -8 
Epi.    8 score:  -18 
Epi.    9 score:   -8 
Epi.   10 score:  -15 
Epi.   11 score:  -21 
Epi.   12 score:  -15 
Epi.   13 score:  -12 
Epi.   14 score:  -16 
Epi.   15 score:   -9 
Epi.   16 score:  -21 
Epi.   17 score:  -17 
Epi.   18 score:  -10 
Epi.   19 score:  -18 
Epi.   20 score:  -33 
Epi.   21 score:  -25 
Epi.   22 score:  -19 
Epi.   23 score:  -10 
Epi.   24 score:  -12 
Epi.   25 score:  -24 
Epi.   26 score:  -11 
Epi.   27 score:  -12 
Epi.   28 score:  -15 
Epi.   29 score:  -12 
Epi.   30 score:  -11 
Epi.   31 score:  -32 
Epi.   32 score:  -11 
Epi.   33 score:   -9 
Epi.   34 score:  -12 
Epi.   35 score:  -12 
Epi.   36 score:  -13 
Epi.   37 score:  -16 
Epi.   38 score:   -7 
Epi.   39 score:  -18 
Epi.   40 score:  -10 
Epi.   41 score:  -18 
Epi.   42 score:  -13 
Epi.   43 score:  -12 
Epi.   44 s

Epi.  362 score:  -20 
Epi.  363 score:  -24 
Epi.  364 score:  -10 
Epi.  365 score:  -11 
Epi.  366 score:   -9 
Epi.  367 score:  -12 
Epi.  368 score:  -13 
Epi.  369 score:  -11 
Epi.  370 score:  -24 
Epi.  371 score:  -24 
Epi.  372 score:  -16 
Epi.  373 score:   -9 
Epi.  374 score:   -7 
Epi.  375 score:  -28 
Epi.  376 score:   -7 
Epi.  377 score:  -10 
Epi.  378 score:   -7 
Epi.  379 score:   -9 
Epi.  380 score:   -9 
Epi.  381 score:  -12 
Epi.  382 score:   -8 
Epi.  383 score:  -15 
Epi.  384 score:  -10 
Epi.  385 score:  -20 
Epi.  386 score:  -26 
Epi.  387 score:  -13 
Epi.  388 score:   -8 
Epi.  389 score:   -8 
Epi.  390 score:  -15 
Epi.  391 score:  -28 
Epi.  392 score:  -23 
Epi.  393 score:   -7 
Epi.  394 score:  -19 
Epi.  395 score:   -9 
Epi.  396 score:  -16 
Epi.  397 score:  -32 
Epi.  398 score:  -12 
Epi.  399 score:  -28 
Epi.  400 score:  -23 
Epi.  401 score:  -16 
Epi.  402 score:   -9 
Epi.  403 score:  -22 
Epi.  404 score:   -7 
Epi.  405 s

Epi.  568 score: -519 E[L]:   30.9 e: 0.0100 norm: 3550 actions:   1 233   1 3
Epi.  569 score: -488 E[L]:   41.4 e: 0.0100 norm: 4129 actions:   0 227   1 3
Epi.  570 score: -493 E[L]:   38.1 e: 0.0100 norm: 4061 actions:   0 228   1 3
Epi.  571 score: -472 E[L]:   29.8 e: 0.0100 norm: 3678 actions:   0 221   4 3
Epi.  572 score: -479 E[L]:   38.4 e: 0.0100 norm: 4236 actions:   0 225   1 3
Epi.  573 score: -472 E[L]:   37.6 e: 0.0100 norm: 4218 actions:   1 221   3 3
Epi.  574 score: -479 E[L]:   37.3 e: 0.0100 norm: 4255 actions:   0 225   1 3
Epi.  575 score: -500 E[L]:   35.5 e: 0.0100 norm: 4184 actions:   0 229   2 3
Epi.  576 score: -506 E[L]:   33.4 e: 0.0100 norm: 4201 actions:   1 229   2 3
Epi.  577 score: -514 E[L]:   31.3 e: 0.0100 norm: 4063 actions:   3 231   1 3
Epi.  578 score: -529 E[L]:   27.6 e: 0.0100 norm: 3928 actions:   0 236   1 3
Epi.  579 score: -470 E[L]:   31.6 e: 0.0100 norm: 4246 actions:   1 220   4 3
Epi.  580 score: -480 E[L]:   30.0 e: 0.0100 norm: 4

Epi.  672 score: -232 E[L]:    1.4 e: 0.0100 norm: 1356 actions:   7  95 134 3
Epi.  673 score: -224 E[L]:    1.4 e: 0.0100 norm: 1291 actions:  11  92 124 3
Epi.  674 score: -227 E[L]:    1.4 e: 0.0100 norm: 1260 actions:   7  95 131 3
Epi.  675 score: -214 E[L]:    1.4 e: 0.0100 norm: 1241 actions:   8  95 128 3
Epi.  676 score: -223 E[L]:    1.4 e: 0.0100 norm: 1202 actions:   9  94 127 3
Epi.  677 score: -201 E[L]:    1.5 e: 0.0100 norm: 1212 actions:   9  98 123 3
Epi.  678 score: -206 E[L]:    1.5 e: 0.0100 norm: 1180 actions:   8  96 127 3
Epi.  679 score: -205 E[L]:    1.5 e: 0.0100 norm: 1156 actions:  11  95 121 3
Epi.  680 score: -220 E[L]:    1.6 e: 0.0100 norm: 1162 actions:   9  95 124 3
Epi.  681 score: -221 E[L]:    1.6 e: 0.0100 norm: 1148 actions:   9  94 128 3
Epi.  682 score: -216 E[L]:    1.7 e: 0.0100 norm: 1157 actions:   8  95 130 3
Epi.  683 score: -207 E[L]:    1.7 e: 0.0100 norm: 1123 actions:   8  94 131 3
Epi.  684 score: -211 E[L]:    1.7 e: 0.0100 norm: 1

Epi.  777 score: -218 E[L]:   14.8 e: 0.0100 norm: 927 actions:  18  59 161 3
Epi.  778 score: -249 E[L]:   13.0 e: 0.0100 norm: 928 actions:  19  51 161 3
Epi.  779 score: -238 E[L]:   14.0 e: 0.0100 norm: 1019 actions:  19  52 155 3
Epi.  780 score: -236 E[L]:   12.9 e: 0.0100 norm: 954 actions:  17  53 164 3
Epi.  781 score: -234 E[L]:   12.9 e: 0.0100 norm: 931 actions:  17  52 157 3
Epi.  782 score: -233 E[L]:   14.6 e: 0.0100 norm: 904 actions:  17  51 159 3
Epi.  783 score: -248 E[L]:   13.9 e: 0.0100 norm: 1029 actions:  16  50 167 3
Epi.  784 score: -239 E[L]:   13.8 e: 0.0100 norm: 862 actions:  20  49 159 3
Epi.  785 score: -234 E[L]:   13.9 e: 0.0100 norm: 882 actions:  18  54 166 3
Epi.  786 score: -239 E[L]:   13.5 e: 0.0100 norm: 1010 actions:  17  50 165 3
Epi.  787 score: -219 E[L]:   12.6 e: 0.0100 norm: 956 actions:  17  55 161 3
Epi.  788 score: -216 E[L]:   11.3 e: 0.0100 norm: 1024 actions:  18  53 159 3
Epi.  789 score: -213 E[L]:   12.8 e: 0.0100 norm: 886 actio

...

Epi. 4210 score: -401 E[L]:  101.7 e: 0.0100 norm: 3315 actions:   7  17 213 3
Epi. 4211 score: -369 E[L]:   99.6 e: 0.0100 norm: 2971 actions:   8  14 205 3
Epi. 4212 score: -409 E[L]:  103.4 e: 0.0100 norm: 3205 actions:   3  16 219 3
Epi. 4213 score: -399 E[L]:  110.4 e: 0.0100 norm: 3450 actions:   6  14 213 3
Epi. 4214 score: -388 E[L]:   99.0 e: 0.0100 norm: 3432 actions:   9  16 207 3
Epi. 4215 score: -418 E[L]:  102.7 e: 0.0100 norm: 3400 actions:   5  13 218 3
Epi. 4216 score: -377 E[L]:   96.5 e: 0.0100 norm: 3296 actions:   6  14 207 3
Epi. 4217 score: -360 E[L]:   95.5 e: 0.0100 norm: 3430 actions:   6  16 204 3
Epi. 4218 score: -391 E[L]:  108.3 e: 0.0100 norm: 3103 actions:   6  16 211 3
Epi. 4219 score: -370 E[L]:  109.3 e: 0.0100 norm: 3287 actions:  10  15 202 3
Epi. 4220 score: -397 E[L]:  110.9 e: 0.0100 norm: 3276 actions:  10  13 211 3
Epi. 4221 score: -387 E[L]:  110.6 e: 0.0100 norm: 3686 actions:   6  13 207 3
Epi. 4222 score: -390 E[L]:  111.2 e: 0.0100 norm: 3

Epi. 4314 score: -401 E[L]:  108.1 e: 0.0100 norm: 3588 actions:   7  14 211 3
Epi. 4315 score: -373 E[L]:   97.6 e: 0.0100 norm: 3277 actions:   5  14 207 3
Epi. 4316 score: -374 E[L]:  106.1 e: 0.0100 norm: 3217 actions:   9  12 204 3
Epi. 4317 score: -405 E[L]:  107.7 e: 0.0100 norm: 3530 actions:   8  12 212 3
Epi. 4318 score: -404 E[L]:   98.3 e: 0.0100 norm: 3283 actions:   6  10 210 3
Epi. 4319 score: -401 E[L]:  108.9 e: 0.0100 norm: 3733 actions:   7  15 211 3
Epi. 4320 score: -387 E[L]:  103.5 e: 0.0100 norm: 3462 actions:   6  12 209 3
Epi. 4321 score: -385 E[L]:  100.6 e: 0.0100 norm: 3569 actions:   7  12 207 3
Epi. 4322 score: -387 E[L]:  111.3 e: 0.0100 norm: 3847 actions:   7  14 207 3
Epi. 4323 score: -407 E[L]:  102.9 e: 0.0100 norm: 3393 actions:   8  13 213 3
Epi. 4324 score: -378 E[L]:  102.3 e: 0.0100 norm: 3508 actions:   7  15 206 3
Epi. 4325 score: -399 E[L]:   99.2 e: 0.0100 norm: 3049 actions:   6  14 211 3
Epi. 4326 score: -383 E[L]:   98.3 e: 0.0100 norm: 3

Epi. 4418 score: -384 E[L]:   95.6 e: 0.0100 norm: 3351 actions:   7  13 206 3
Epi. 4419 score: -395 E[L]:   91.2 e: 0.0100 norm: 3088 actions:   9  13 210 3
Epi. 4420 score: -394 E[L]:  104.2 e: 0.0100 norm: 3474 actions:   7  12 210 3
Epi. 4421 score: -415 E[L]:  112.7 e: 0.0100 norm: 3454 actions:   6  15 217 3
Epi. 4422 score: -399 E[L]:  108.9 e: 0.0100 norm: 4219 actions:   4  15 215 3
Epi. 4423 score: -379 E[L]:  108.2 e: 0.0100 norm: 3689 actions:   6  13 207 3
Epi. 4424 score: -369 E[L]:  105.3 e: 0.0100 norm: 3283 actions:   5  16 205 3
Epi. 4425 score: -382 E[L]:  103.9 e: 0.0100 norm: 3310 actions:   6  13 208 3
Epi. 4426 score: -399 E[L]:  105.6 e: 0.0100 norm: 3577 actions:   4  15 215 3
Epi. 4427 score: -400 E[L]:   84.6 e: 0.0100 norm: 3139 actions:   7  13 212 3
Epi. 4428 score: -398 E[L]:  108.9 e: 0.0100 norm: 3881 actions:   5  16 211 3
Epi. 4429 score: -372 E[L]:   97.2 e: 0.0100 norm: 3305 actions:   5  16 206 3
Epi. 4430 score: -366 E[L]:  110.4 e: 0.0100 norm: 3

Epi. 4522 score: -395 E[L]:  100.5 e: 0.0100 norm: 3571 actions:   5  12 209 3
Epi. 4523 score: -390 E[L]:  112.8 e: 0.0100 norm: 3747 actions:   8  13 208 3
Epi. 4524 score: -417 E[L]:  111.9 e: 0.0100 norm: 4024 actions:   5  14 218 3
Epi. 4525 score: -390 E[L]:  114.8 e: 0.0100 norm: 3497 actions:   7  12 209 3
Epi. 4526 score: -381 E[L]:  111.8 e: 0.0100 norm: 3812 actions:   5  13 207 3
Epi. 4527 score: -392 E[L]:  105.9 e: 0.0100 norm: 3308 actions:   6  11 209 3
Epi. 4528 score: -423 E[L]:  105.0 e: 0.0100 norm: 3769 actions:   4  14 220 3
Epi. 4529 score: -389 E[L]:   93.7 e: 0.0100 norm: 3197 actions:   8  13 208 3
Epi. 4530 score: -387 E[L]:   92.9 e: 0.0100 norm: 3127 actions:   6  12 209 3
Epi. 4531 score: -382 E[L]:   98.3 e: 0.0100 norm: 3821 actions:   5  13 208 3
Epi. 4532 score: -390 E[L]:  105.6 e: 0.0100 norm: 3516 actions:   5  14 211 3
Epi. 4533 score: -380 E[L]:  124.9 e: 0.0100 norm: 3924 actions:   5  13 208 3
Epi. 4534 score: -378 E[L]:  105.1 e: 0.0100 norm: 3

KeyboardInterrupt: 

In [1]:
k = list(map(lambda x: x.grad, net.parameters()))
l = list(map(lambda x: x.data, net.parameters()))
fig = plt.figure(figsize=(15, 15))
fig.suptitle('Pretty well trained', fontsize=25)

ax = plt.subplot(211)
plt.hist([
    k[0].cpu().numpy().reshape(-1),
    k[2].cpu().numpy().reshape(-1),
], bins=20, color=['r', 'orange'], label=['Layer 1 (input)', 'Layer 2 (output)'])
ax.set_title(r'Gradients $\partial \mathcal{L} / \partial w$', fontsize=22)
ax.legend(fontsize=15)

ax = plt.subplot(212)
plt.hist([
    l[0].cpu().numpy().reshape(-1),
    l[2].cpu().numpy().reshape(-1),
], bins=20, color=['r', 'orange'], label=['Layer 1 (input)', 'Layer 2 (output)'])
ax.set_title('Weights', fontsize=22)
ax.legend(fontsize=15)

plt.show()

NameError: name 'net' is not defined

In [2]:
fig, host = plt.subplots()
fig.set_size_inches(15, 15)
fig.subplots_adjust(right=.75)

par1 = host.twinx()
# par2 = host.twinx()

# par2.spines['right'].set_position(('axes', 1.2))
# par2.spines['right'].set_visible(True)

p1, = host.plot(record['score'], 'r', label='score')
p2, = par1.plot(record['average loss'], 'orange', label='average loss')

host.set_xlabel('episode', fontsize=22)
host.set_ylabel('score', fontsize=22)
par1.set_ylabel('average loss', fontsize=22)

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())

tkw = dict(size=4, width=1.5)
host.tick_params(axis='x', **tkw) 
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)

lines = [p1, p2]
host.legend(lines, [l.get_label() for l in lines], fontsize=20)  # 이렇게 안 하면 child에 그려진 애들은 legend가 안 붙나봄
plt.show() 

NameError: name 'plt' is not defined

In [ ]:
# r: range
# vc: closing velocity
# los: line of sight angle
# daz: azimuthal rate (horizontal look angle, right +)
# dlos: los rate
print('Env Reset')
obs0 = env.reset() 
print('observations')
for label, val in zip(['range', 'vc', 'los', 'daz', 'dlos'], obs0):
    print(f'  {label:8}:\t{val}')

In [ ]:
print('Env Get Started')
obs, reward, done, info = env.step(a)
a =  np.argmax(net(torch.from_numpy(obs).float().to(device)).detach().cpu())
print('action:\t', a.item()) 
print('reward:\t', reward)
print('done:\t', done) 
print('observations')
for label, val in zip(['range', 'vc', 'los', 'daz', 'dlos'], obs):
    print(f'  {label:10}:\t{val}')

print('\ninfos')
for label, val in zip(['hdot_cmd', 'range', 'elev', 'azim', 'Pm_NED', 'Pt_NED', 'h'], info):
    print(f'  {label:10}:\t{val}')